In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import torch; torch.cuda.device_count()

1

In [3]:
from cnlpt.train_system import main, ModelArguments, CnlpTrainingArguments
from cnlpt.cnlp_data import DataTrainingArguments
from cnlpt.api.negation_rest import create_instance_string

from transformers import logging, AutoTokenizer, HfArgumentParser
from bertviz import model_view

import os

In [4]:
logging.set_verbosity_error()  # Remove line to see warnings

In [10]:
args = (
#    "--do_train "
    "--task_name polarity "
    f"--data_dir {os.path.expanduser('~/mnt/r/DeepLearning/mmtl/polarity/seed/')} "
    "--model_name_or_path roberta-base "
    "--output_dir temp/ "
    "--overwrite_output_dir "
    "--cache cache "
    "--num_train_epochs 3 "
    "--learning_rate 1e-5 "
    "--gradient_accumulation_steps 4 "
    "--per_device_train_batch_size 32 "
    "--layer 12"
).split()

In [11]:
args

['--task_name',
 'polarity',
 '--data_dir',
 '/home/angus/mnt/r/DeepLearning/mmtl/polarity/seed/',
 '--model_name_or_path',
 'roberta-base',
 '--output_dir',
 'temp/',
 '--overwrite_output_dir',
 '--cache',
 'cache',
 '--num_train_epochs',
 '3',
 '--learning_rate',
 '1e-5',
 '--gradient_accumulation_steps',
 '4',
 '--per_device_train_batch_size',
 '32',
 '--layer',
 '12']

In [13]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, CnlpTrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)

In [14]:
model_args.output_attentions

False

In [16]:
training_args.do_train

False

In [17]:
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    add_prefix_space=True,
    additional_special_tokens=['<e>', '</e>', '<a1>', '</a1>', '<a2>', '</a2>', '<cr>', '<neg>']
)

In [18]:
model = main(model_args, data_args, training_args)

12/28/2021 22:22:49 - WARNING - cnlpt.train_system -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/28/2021 22:22:49 - INFO - cnlpt.train_system -   Training/evaluation parameters CnlpTrainingArguments(output_dir='temp/', overwrite_output_dir=True, do_train=False, do_eval=None, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=4, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, logging_dir='runs/Dec28_22-22-30_nlp-gpu', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, save_strategy

In [19]:
model

CnlpRobertaForClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50273, 768)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [22]:
inst_str = create_instance_string(
    'The patient has a sore knee and headache but denies nausea and has no anosmia.',
    [18, 27]
)
print(inst_str)
inputs = tokenizer.encode(inst_str, return_tensors='pt').to('cuda')
outputs = model(inputs)
print(outputs)

The patient has a  <e> sore knee </e>  and headache but denies nausea and has no anosmia.
SequenceClassifierOutput(loss=None, logits=[tensor([[-0.0216,  0.0004]], device='cuda:0', grad_fn=<AddmmBackward>)], hidden_states=None, attentions=None)


In [ ]:
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0])
model_view(attention, tokens)